In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from datasets import load_dataset

from ultralytics import SAM
from skimage import io
import os
import csv

In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [35]:
dataset = load_dataset("e1010101/tongue-images-384-no-augmentation")

In [26]:
model = SAM("sam2_l.pt")
model.info()

Model summary: 854 layers, 224,430,130 parameters, 224,430,130 gradients


(854, 224430130, 224430130, 0.0)

In [36]:
def segment(image):    
    image_np = np.array(image)
    
    # Segmentation
    results_pil = model(image, points=[[350, 320], [0, 0]], labels=[1, 0])
    
    # Get the mask from the results
    mask_pil = results_pil[0].masks.data[0].cpu().numpy()
    
    # Masking
    binary_mask = mask_pil > 0.5
    rgb_mask = np.repeat(binary_mask[:, :, np.newaxis], 3, axis=2)
    segmented_image = image_np * rgb_mask
    
    return segmented_image

In [37]:
all_labels = ["Crack", "Red-Dots", "Toothmark"]

In [29]:
train_csv = "train_labels.csv"
valid_csv = "valid_labels.csv"
test_csv = "test_labels.csv"

In [38]:
for df in [train_csv, valid_csv, test_csv]:
    with open(df, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        header = ['filename'] + all_labels
        csvwriter.writerow(header)

In [31]:
output_train = "output/train_segmented/"
output_valid = "output/valid_segmented/"
output_test = "output/test_segmented/"

In [32]:
output_original_train = "output/train/"
output_original_valid = "output/valid/"
output_original_test = "output/test/"

In [33]:
def segment_dataset(dir_segment, dir_original, split, csv_towrite):
    for idx, item in enumerate(dataset[split]):
        segmented_image = segment(item['image'])
        segmented_image_output = Image.fromarray(segmented_image, mode='RGB')
        output_filename = f"segmented_{idx}.png"
        output_path = os.path.join(dir_segment, output_filename)
        segmented_image_output.save(output_path)
        
        original_filename = f"original_{idx}.png"
        output_path = os.path.join(dir_original, original_filename)
        item['image'].save(output_path)
        
        row = [output_filename] + item['labels']
        with open(csv_towrite, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(row)

In [39]:
segment_dataset(output_train, output_original_train, 'train', train_csv)
segment_dataset(output_valid, output_original_valid, 'validation', valid_csv)
segment_dataset(output_test, output_original_test, 'test', test_csv)


0: 1024x1024 1 0, 1 1, 213.2ms
Speed: 103.1ms preprocess, 213.2ms inference, 0.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1 0, 1 1, 177.5ms
Speed: 6.0ms preprocess, 177.5ms inference, 0.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1 0, 1 1, 175.0ms
Speed: 5.0ms preprocess, 175.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1 0, 1 1, 181.0ms
Speed: 5.0ms preprocess, 181.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1 0, 1 1, 173.6ms
Speed: 5.0ms preprocess, 173.6ms inference, 0.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1 0, 1 1, 180.1ms
Speed: 5.0ms preprocess, 180.1ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1 0, 1 1, 181.0ms
Speed: 7.0ms preprocess, 181.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1 0, 1 1, 179.4ms
Speed: 5.0ms preprocess, 179.4ms inference